# **Objetivo del notebook**

En esta etapa ya hemos decidido el tipo de limpieza más óptima, y hemos elegido nuestra variable objetivo.

Cargaremos el dataset que limpiamos en Cleaning.ipynb, y probaremos distintos parámetros en tfidf y en algún modelo de ensemble para intentar conseguir un overfitting inferior al 5%.



# **Contenido del Notebook**

1. Búsqueda de hiperparámetros (pruebas)
2. Pipeline con vectorización TF-IDF y modelo
3. Métricas
4. Conclusiones

#### **Instalaciones por si ejecutas el notebook desde deepnote, o google colab y no creas tu propio entorno virtual**

Por defecto las dejamos comentadas. Quitar el '#' si se desea usarlas.

In [ ]:
#!pip install optuna > /dev/null 2>&1

In [ ]:
#!pip xgboost > /dev/null 2>&1

In [ ]:
# Data analysis libraries
import pandas as pd

# Machine learning libraries
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score
import pickle
import optuna


In [ ]:
# read csv file
path = "/content/cleaned_youtoxic.csv"
df = pd.read_csv(path)

### **1. Pipeline con vectorización TF-IDF y modelo**

In [ ]:
# Preparación de datos para el modelo
X = df['Text']
y = df['IsToxic']

In [ ]:
# Creamos un pipeline con TfidfVectorizer y XGBClassifier donde iremos actualizando los hiperparámetros que saquemos más abajo
pipeline_model = Pipeline([
    ('vectorizer', TfidfVectorizer(max_features=4000, ngram_range=(1, 3), min_df=3, max_df=0.1, sublinear_tf=True, stop_words='english')),
    ('classifier', XGBClassifier(learning_rate=0.04, subsample=0.8, max_depth=3, n_estimators=100))
])

In [ ]:
# División de los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Entrenamos el modelo con el pipeline
pipeline_model.fit(X_train, y_train)

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(max_df=0.1, max_features=4000, min_df=3,
                                 ngram_range=(1, 3), stop_words='english',
                                 sublinear_tf=True)),
                ('classifier',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None, device=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, eval...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.04,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=3, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=100, n_jobs=None,
                               num_parallel_tree=None, random_state=None, ...))])

### **2. Métricas**

In [ ]:
# Realizamos predicciones en conjunto de prueba
y_test_pred = pipeline_model.predict(X_test)

# Realizamos predicciones en conjunto de entrenamiento
y_train_pred = pipeline_model.predict(X_train)

In [ ]:
# Imprimimos el classification report y accuracy en conjunto de prueba
print("Classification Report en conjunto de prueba:")
print(classification_report(y_test, y_test_pred))
print(f"Accuracy en conjunto de prueba: {accuracy_score(y_test, y_test_pred):.4f}")

# Calculamos y imprimimos el accuracy en conjunto de entrenamiento
accuracy_train = accuracy_score(y_train, y_train_pred)
print(f"Accuracy en conjunto de entrenamiento: {accuracy_train:.4f}")

# Calculamos e imprimimos el porcentaje de overfitting
overfitting_percentage = (accuracy_train - accuracy_score(y_test, y_test_pred)) * 100
print(f"Porcentaje de Overfitting: {overfitting_percentage:.2f}%")

Classification Report en conjunto de prueba:
              precision    recall  f1-score   support

       False       0.65      0.94      0.77       108
        True       0.84      0.41      0.55        92

    accuracy                           0.69       200
   macro avg       0.75      0.67      0.66       200
weighted avg       0.74      0.69      0.67       200

Accuracy en conjunto de prueba: 0.6950
Accuracy en conjunto de entrenamiento: 0.7422
Porcentaje de Overfitting: 4.72%


**Mejores métricas**

Accuracy en conjunto de prueba: 0.6950

Accuracy en conjunto de entrenamiento: 0.7422

Porcentaje de Overfitting: 4.72%

**Mejores parámetros:**

- De tfidf:

max_features=4000, ngram_range=(1, 3), min_df=3, max_df=0.1, sublinear_tf=True, stop_words='english'

- De XGBClassifier:

learning_rate=0.04, subsample=0.8, max_depth=3, n_estimators=100

#### **Guardamos en un pickle el modelo definitivo**

Lo dejamos comentado hasta que tengamos el modelo definitivo

In [ ]:
#pickle.dump(pipeline_model, open('modelo.pkl', 'wb'))

### **3. Búsqueda de hiperparámetros (pruebas)**

#### **3.1. Búsqueda de hiperparámetros sólo en TF-IDF**

In [ ]:
# Preparación de datos para el modelo
X = df['Text']
y = df['IsToxic']

# División de los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Define el espacio de búsqueda para los hiperparámetros de tfidf
def objective(trial):
    # Define el pipeline con TfidfVectorizer y GradientBoostingClassifier
    pipeline_model = Pipeline([
        ('vectorizer', TfidfVectorizer(
            max_features=trial.suggest_categorical('max_features', [500, 1000, 4000,5000]),
            ngram_range=trial.suggest_categorical('ngram_range', [(1, 1), (1, 2), (1, 3), (1, 4)]),
            min_df=trial.suggest_int('min_df', 1, 5),
            max_df=trial.suggest_float('max_df', 0.8, 1.0),
            sublinear_tf=trial.suggest_categorical('sublinear_tf', [False, True]),
            stop_words=trial.suggest_categorical('stop_words', [None, 'english'])
        )),
        ('classifier', XGBClassifier())
    ])

    # Entrenar el modelo
    pipeline_model.fit(X_train, y_train)

    # Realizar predicciones en conjunto de prueba
    y_test_pred = pipeline_model.predict(X_test)

    # Calcular y devolver la inversa de la precisión como valor a minimizar
    return 1.0 / accuracy_score(y_test, y_test_pred)

# Configuración de Optuna
study = optuna.create_study(direction='minimize')  # Queremos minimizar la inversa de la precisión
study.optimize(objective, n_trials=1000)  # Ajusta n_trials según tus recursos

# Imprimir los mejores parámetros encontrados
print("Mejores parámetros encontrados:")
print(study.best_params)

**Mejores parámetros:**

max_features=4000,

ngram_range=(1, 3),

min_df=3, max_df=0.1,

sublinear_tf=True,

stop_words='english'

#### **3.2. Búsqueda de hiperparámetros sólo en XGBClassifier**

In [ ]:

# Preparación de datos para el modelo
X = df['Text']
y = df['IsToxic']

# División de los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Define el espacio de búsqueda para los hiperparámetros del GradientBoostingClassifier
def objective(trial):
    # Define el pipeline con TfidfVectorizer y GradientBoostingClassifier
    pipeline_model = Pipeline([
        ('vectorizer', TfidfVectorizer(
            max_features=4000,
            ngram_range=(1, 3),
            min_df=3, max_df=0.1,
            sublinear_tf=True,
            stop_words='english'
        )),
        ('classifier', XGBClassifier(
            learning_rate=trial.suggest_float('learning_rate', 0.01, 0.2, log=True),
            subsample=trial.suggest_float('subsample', 0.6, 1.0),
            max_depth=trial.suggest_int('max_depth', 3, 7),
            n_estimators=trial.suggest_categorical('n_estimators', [50, 100, 200])
        )),
    ])

    # Entrenar el modelo
    pipeline_model.fit(X_train, y_train)

    # Realizar predicciones en conjunto de prueba
    y_test_pred = pipeline_model.predict(X_test)

    # Calcular y devolver la inversa de la precisión como valor a minimizar
    return 1.0 / accuracy_score(y_test, y_test_pred)

# Configuración de Optuna
study = optuna.create_study(direction='minimize')  # Queremos minimizar la inversa de la precisión
study.optimize(objective, n_trials=1000)  # Ajusta n_trials según tus recursos

# Imprimir los mejores parámetros encontrados
print("Mejores parámetros encontrados:")
print(study.best_params)

**Mejores parámetros:**

learning_rate=0.04,

subsample=0.8,

max_depth=3,

n_estimators=100

#### **3.3. Búsqueda de hiperparámetros en TF-IDF y XGBClassifier a la vez**

In [ ]:
# Supongamos que tienes un DataFrame df con columnas 'Text' y 'IsToxic'

# Define el espacio de búsqueda para los hiperparámetros
def objective(trial):
    # Preparación de datos para el modelo
    X = df['Text']
    y = df['IsToxic']

    # División de los datos en conjunto de entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # Define el pipeline con TfidfVectorizer y GradientBoostingClassifier
    class LogUniformDistribution(optuna.distributions.UniformDistribution):
        def to_external_repr(self, x):
            return np.exp(x)

        def to_internal_repr(self, x):
            return np.log(x)

    pipeline_model = Pipeline([
        ('vectorizer', TfidfVectorizer(
            max_features=trial.suggest_categorical('max_features', [500, 1000, 4000,5000]),
            ngram_range=trial.suggest_categorical('ngram_range', [(1, 1), (1, 2), (1, 3), (1, 4)]),
            min_df=trial.suggest_int('min_df', 1, 5),
            max_df=trial.suggest_float('max_df', 0.8, 1.0),
            sublinear_tf=trial.suggest_categorical('sublinear_tf', [False, True]),
            stop_words=trial.suggest_categorical('stop_words', [None, 'english'])
        )),
        ('classifier', XGBClassifier(
            learning_rate=trial.suggest_float('learning_rate', 0.01, 0.2, log=True),
            subsample=trial.suggest_float('subsample', 0.6, 1.0),
            max_depth=trial.suggest_int('max_depth', 3, 7),
            n_estimators=trial.suggest_categorical('n_estimators', [50, 100, 200])
        )),
    ])

    # Entrenar el modelo
    pipeline_model.fit(X_train, y_train)

    # Realizar predicciones en conjunto de prueba
    y_test_pred = pipeline_model.predict(X_test)

    # Calcular y devolver la inversa de la precisión como valor a minimizar
    return 1.0 / accuracy_score(y_test, y_test_pred)

# Configuración de Optuna
study = optuna.create_study(direction='minimize')  # Queremos minimizar la inversa de la precisión
study.optimize(objective, n_trials=1000)  # Ajusta n_trials según tus recursos

# Obtener el mejor modelo
best_model = study.best_trial.user_attrs.get('best_model', None)

# Imprimir los mejores parámetros encontrados
print("Mejores parámetros encontrados:")
print(study.best_params)


### **4. Conclusiones**

Al tener tan pocos datos es difícil reducir el overfitting. Ajustando los hiperparámetros hemos conseguido bajarlo a 4.72%. Si subíamos la accuracy, el overfitting seguía siendo alto, así que optamos por la opción que más bajaba el overfitting sin perjudicar demasiado al accuracy.

Se nos ocurren 2 alternativas:

Una opción interesante de probar sería aumentar el conjunto de datos uniendo el dataset inicial a algún otro que contenga mensajes de odio.

Otra opción sería probar algún modelo preentrenado de hugging face, de manera que la escasez de datos no afecte al entrenamiento.